<a href="https://colab.research.google.com/github/tmss1212/backtesting/blob/main/backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#ライブラリインストール
!pip install backtesting numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=36a4ec08cf7b4b870ede9576ec224e3cbfdebc24b24c5319c9e1a6e2a2d5582d
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [3]:
#データ読み込み
import pandas as pd
import datetime as dt
import numpy as np
import pandas_datareader.data as web

code = '8035.JP'
start = '2013-05-01'
end = '2023-10-30'

df_stooq = web.DataReader(code, 'stooq', start, end)
df_stooq.sort_index(inplace=True)
display(df_stooq.head(3))
display(df_stooq.tail(3))

,Open,High,Low,Close,Volume
Date,,,,,
2013-05-01,1333.01,1344.00,1302.77,1315.14,9.965559e+06
2013-05-02,1309.65,1312.39,1278.04,1283.53,7.555123e+06
2013-05-07,1341.25,1379.73,1327.51,1370.11,8.982836e+06


,Open,High,Low,Close,Volume
Date,,,,,
2023-10-26,19465.0,19590.0,19330.0,19340.0,4055900.0
2023-10-27,19450.0,19705.0,19265.0,19610.0,3323800.0
2023-10-30,19595.0,19900.0,19555.0,19800.0,2405100.0


In [4]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
#トレード戦略クラス
class SmaCross(Strategy):
    #短期取得日数
    n1 = 10
    #長期取得日数
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma_short = self.I(SMA, close, self.n1)
        self.sma_long = self.I(SMA, close, self.n2)

    #売買条件アルゴリズム
    def next(self):
        #ゴールデンクロス
        if crossover(self.sma_short, self.sma_long):
            self.buy()
        #デッドクロス
        elif crossover(self.sma_long, self.sma_short):
            self.sell()

#トレード戦略クラス
bt = Backtest(data=df_stooq,
              strategy=SmaCross,
              cash=3000000,
              commission=.002,
              exclusive_orders=True)

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [5]:
#バックテスト実行
output = bt.run()
print(output)
bt.plot()

Start                     2013-05-01 00:00:00
End                       2023-10-30 00:00:00
Duration                   3834 days 00:00:00
Exposure Time [%]                   99.142634
Equity Final [$]                2813129.78518
Equity Peak [$]                 5588260.56418
Return [%]                          -6.229007
Buy & Hold Return [%]             1405.543136
Return (Ann.) [%]                   -0.629626
Volatility (Ann.) [%]               35.173019
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -63.39256
Avg. Drawdown [%]                  -12.989462
Max. Drawdown Duration     1777 days 00:00:00
Avg. Drawdown Duration      189 days 00:00:00
# Trades                                  141
Win Rate [%]                        42.553191
Best Trade [%]                      42.007492
Worst Trade [%]                    -22.622611
Avg. Trade [%]                    

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1309', ...)

In [7]:
#パラメータ最適化後
output_opt = bt.optimize(n1=range(5, 100, 5),
                         n2=range(5, 100, 5),
                         constraint=lambda p: p.n1 < p.n2,
                         maximize='Return [%]',
                         method='grid',
                         max_tries=None,
                         return_heatmap=False,
                         return_optimization=False,
                         random_state=None)
print(output_opt)
bt.plot()

Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

Start                     2013-05-01 00:00:00
End                       2023-10-30 00:00:00
Duration                   3834 days 00:00:00
Exposure Time [%]                   95.206547
Equity Final [$]                9177830.58916
Equity Peak [$]                10493547.08396
Return [%]                         205.927686
Buy & Hold Return [%]             1405.543136
Return (Ann.) [%]                   11.606971
Volatility (Ann.) [%]               39.889828
Sharpe Ratio                         0.290976
Sortino Ratio                        0.502537
Calmar Ratio                         0.207273
Max. Drawdown [%]                    -55.9984
Avg. Drawdown [%]                   -8.843041
Max. Drawdown Duration      664 days 00:00:00
Avg. Drawdown Duration       70 days 00:00:00
# Trades                                   37
Win Rate [%]                        48.648649
Best Trade [%]                     113.848727
Worst Trade [%]                    -32.457473
Avg. Trade [%]                    

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1989', ...)